In [31]:
import os
import time
#mysys="jupyter" #
mysys="colab"

splt="_9505b_"
sn='sn_'
sprat=0.05

if mysys=="colab":
    from google.colab import drive
    drive.mount('/content/drive')
    path='/content/drive/MyDrive/ANN_Edge/unsw_data/final/'
    slash="/"
    ML_path=path+'ml/'
    STDN_path=ML_path+ '/ml/stdn/'
else:
    current_folder = os.getcwd()
    print(current_folder)
    slash="\\"
    path=current_folder.replace('Model_05', "") + 'ANN_Edge\\final\\'
    print(path)
    ML_path=current_folder.replace('Model_05', "") + 'ANN_Edge\\final\ml\\'
    STDN_path=current_folder.replace('Model_05', "") + 'ANN_Edge\\final\ml\stdn\\'

shallow_model_hp_path=path+ 'shallow_model_ANN_hp301.keras'
deep_model_hp_path=path+ 'deep_model_ANN3_hp301.keras'

final_hybrid_model_path=path+ 'final_hybrid_model.keras'

inference_data_path1=path+ "UNSW_NB15_inference1.csv"
inference_data_path2=path+ "UNSW_NB15_inference2.csv"

metrics_f=ML_path  + 'mlj1_metrics_hybrid_results.csv'

feat_file=path+"unsw-nb15_f.csv"
f_train=path+'unsw-nb15_training' + splt +'.csv'
f_test=path+'unsw-nb15_testing' + splt+ '.csv'
print(f_train)
print(f_test)

print(shallow_model_hp_path)
print(deep_model_hp_path)
print(final_hybrid_model_path)
print(inference_data_path1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ANN_Edge/unsw_data/final/unsw-nb15_training_9505b_.csv
/content/drive/MyDrive/ANN_Edge/unsw_data/final/unsw-nb15_testing_9505b_.csv
/content/drive/MyDrive/ANN_Edge/unsw_data/final/shallow_model_ANN_hp301.keras
/content/drive/MyDrive/ANN_Edge/unsw_data/final/deep_model_ANN3_hp301.keras
/content/drive/MyDrive/ANN_Edge/unsw_data/final/final_hybrid_model.keras
/content/drive/MyDrive/ANN_Edge/unsw_data/final/UNSW_NB15_inference1.csv


In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load preprocessed data for training and testing
df_train = pd.read_csv(f_train)
df_test = pd.read_csv(f_test)
df_train.info()
# Define the features
features = ['ct_state_ttl', 'sload', 'rate', 'sttl', 'smean', 'dload', 'sbytes', 'ct_srv_dst', 'ct_dst_src_ltm', 'dbytes', 'ackdat', 'dttl', 'ct_dst_sport_ltm', 'dmean','ct_srv_src', 'dinpkt', 'tcprtt', 'dur', 'synack', 'sinpkt']
#features2= ['ct_state_ttl', 'sload', 'rate', 'sttl', 'smean', 'dload', 'sbytes', 'ct_srv_dst', 'ct_dst_src_ltm', 'dbytes', 'ackdat', 'dttl', 'ct_dst_sport_ltm', 'dmean', 'ct_srv_src', 'dinpkt', 'tcprtt', 'dur', 'synack', 'sinpkt'], 'proto', 'dport', 'stime']

#df_train[features].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176700 entries, 0 to 176699
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 176700 non-null  int64  
 1   dur                176700 non-null  float64
 2   proto              176700 non-null  object 
 3   service            176700 non-null  object 
 4   state              176700 non-null  object 
 5   spkts              176700 non-null  int64  
 6   dpkts              176700 non-null  int64  
 7   sbytes             176700 non-null  int64  
 8   dbytes             176700 non-null  int64  
 9   rate               176700 non-null  float64
 10  sttl               176700 non-null  int64  
 11  dttl               176700 non-null  int64  
 12  sload              176700 non-null  float64
 13  dload              176700 non-null  float64
 14  sloss              176700 non-null  int64  
 15  dloss              176700 non-null  int64  
 16  si

256 / 0.5
128 /0.1
0.001

nurals 256, 128, 64, 32, 16, 8, 4  deep
hyper

512, shallow

number of weights  / match number of parameters


Hybrid these two

In [33]:
# Select features and target variable for training and testing
X_train = df_train[features].values
y_train = df_train['label'].values

X_test = df_test[features].values
y_test = df_test['label'].values

performance_data=[]

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
import time
import psutil
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

performance_metrics = []
# Define function to measure memory usage
def memory_usage():
    process = psutil.Process()
    return process.memory_info().rss

# Define function to measure CPU usage
def cpu_usage():
    return psutil.cpu_percent()

def get_system_info():
    # Get CPU and memory usage
    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent
    return cpu_usage, memory_usage


# Define function to calculate training and testing time
def calculate_time(start_time):
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time

def evaluate_performance(title, y_test, y_preds, mem_use, cpu_use, pred_time,  prt):
    accuracy = accuracy_score(y_test, y_preds)
    precision = precision_score(y_test, y_preds)
    recall = recall_score(y_test, y_preds)
    f1 = f1_score(y_test, y_preds)
    conf_matrix = confusion_matrix(y_test, y_preds)

    metrics ={
        'data': title,
        'time': time.time(),
        'pred time': pred_time,
        'memory usage':mem_use,
        'cpu usage': cpu_use,
        'accuracy':  accuracy,
        'precision':  precision,
        'recall' : recall,
        'f1_score':   f1,
        'conf_matrix':  conf_matrix
    }

    if prt == 1:
        print(title, "Model Performance: ")
        print ('Time',  time.time())
        print ('Prediction time', pred_time)
        print ('Memory Usage', mem_use)
        print ('CPU usage', cpu_use)
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1-score:", f1)
        print("Confusion Matrix:")
        print(conf_matrix)

    return metrics

# Define function to display performance metrics
def display_metrics(test_loss, test_accuracy, val_accuracy, precision, recall, f1, roc_auc, avg_precision):
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)
    print("Validation Accuracy:", val_accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("ROC-AUC Score:", roc_auc)
    print("Average Precision Score:", avg_precision)

In [35]:
from tensorflow.keras.models import load_model

def load_summary (modelpath):
# Load the model from the specified path
	model = load_model(modelpath)
	model.summary()
	return model

def do_inference(model,if_data,if_labels,n,xprt,xprtp):
  if (xprtp==1):
    print(if_data[:1])
    print("-----")

  inference_set=if_data[:n]

  if xprt==1:
  	print(inference_set)

  predictions = model.predict(if_data[:n])
  thresholded_predictions = (predictions >= 0.5).astype(int)
  if xprtp==1:
    print("Prediction", "Actual")
    for i in range(n):
    	print(thresholded_predictions[i], if_labels[i])

  return thresholded_predictions


In [36]:
from sklearn.metrics import confusion_matrix

def performance_info(Xtest, ytest, n):
    test = Xtest[:n]
    preds = ytest[:n]
    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(test, preds)
    print("-------Confusion Matrix:-------")
    print(conf_matrix)
    correct_predictions = 0
    for i in range(n):
        if test[i] == preds[i]:
            correct_predictions += 1
    print("Correct Predictions:", correct_predictions)
    accuracy = correct_predictions / n

    '''
    true_positive = conf_matrix[1, 1]  # Actual class is positive, and predicted class is positive
    false_positive = conf_matrix[0, 1]  # Actual class is negative, but predicted class is positive
    true_negative = conf_matrix[0, 0]  # Actual class is negative, and predicted class is negative
    false_negative = conf_matrix[1, 0]  # Actual class is positive, but predicted class is negative
    '''
    acc = (conf_matrix[0, 0] + conf_matrix[1, 1]) / (conf_matrix[0, 0] + conf_matrix[0, 1] + conf_matrix[1, 0] + conf_matrix[1, 1])
    print("Accuracy : ", accuracy , acc)
    return conf_matrix, acc

def calculate_performance2(conf_matrix):
    # Calculate True Positives (TP), False Positives (FP), True Negatives (TN), False Negatives (FN)
    tp = conf_matrix[1, 1]
    fp = conf_matrix[0, 1]
    tn = conf_matrix[0, 0]
    fn = conf_matrix[1, 0]

    # Calculate Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Calculate Precision
    if tp + fp == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)

    # Calculate Recall
    if tp + fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)

    # Calculate F1 Score
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * (precision * recall) / (precision + recall)

    # Create dictionary to store performance data
    performance_data = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return performance_data


In [37]:
shallow_model=load_summary(shallow_model_hp_path)
n=10
predictions=do_inference(shallow_model, X_test,y_test, n, 0, 0)
#print(predictions)
conf_matrix, accuracy= performance_info(y_test, predictions, n)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 512)               10752     
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 11265 (44.00 KB)
Trainable params: 11265 (44.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
1/1 [==============================] - 0s 67ms/step
-------Confusion Matrix:-------
[[5 0]
 [0 5]]
Correct Predictions: 10
Accuracy :  1.0 1.0


In [38]:
deep_model=load_summary(deep_model_hp_path)
n=10
predictions=do_inference(deep_model, X_test,y_test, n, 0, 0)
#print(predictions, X_test[:n])
conf_matrix, accuracy= performance_info(y_test, predictions, n)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 256)               5376      
                                                                 
 dropout_23 (Dropout)        (None, 256)               0         
                                                                 
 dense_29 (Dense)            (None, 128)               32896     
                                                                 
 dropout_24 (Dropout)        (None, 128)               0         
                                                                 
 dense_30 (Dense)            (None, 64)                8256      
                                                                 
 dropout_25 (Dropout)        (None, 64)                0         
                                                                 
 dense_31 (Dense)            (None, 32)               

## Weighted Average Hybrid

In [39]:
import numpy as np
from keras.models import load_model

start_time = time.time()

# Load models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Make predictions on the test set
preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)

# Define weights for each model
weight1 = 0.3  # Adjust according to preference
weight2 = 0.7  # Adjust according to preference

# Combine predictions using weighted average
hybrid_preds = weight1 * preds1 + weight2 * preds2

# Convert hybrid predictions to labels if necessary
# Assuming binary classification, you can use a threshold of 0.5
hybrid_labels = (hybrid_preds > 0.5).astype(int)

cpu_use, mem_use = get_system_info()
pred_time=calculate_time(start_time)

# Evaluate hybrid model performance
hybrid_accuracy = accuracy_score(y_test, hybrid_labels)
print("Hybrid Model Accuracy:", hybrid_accuracy)
metrics = evaluate_performance("Weighted Average Hybrid .3 : .7 numpy", y_test, hybrid_labels, mem_use, cpu_use, pred_time, prt=1)
print(metrics )
performance_data.append(metrics)

291/291 [==============================] - 1s 2ms/step
Hybrid Model Accuracy: 0.9319354838709677
Weighted Average Hybrid .3 : .7 numpy Model Performance: 
Time 1710341412.8277814
Prediction time 1.8805198669433594
Memory Usage 13.1
CPU usage 22.1
Accuracy: 0.9319354838709677
Precision: 0.9459580169718624
Recall: 0.915496001728982
F1-score: 0.9304777594728171
Confusion Matrix:
[[4431  242]
 [ 391 4236]]
{'data': 'Weighted Average Hybrid .3 : .7 numpy', 'time': 1710341412.8269904, 'pred time': 1.8805198669433594, 'memory usage': 13.1, 'cpu usage': 22.1, 'accuracy': 0.9319354838709677, 'precision': 0.9459580169718624, 'recall': 0.915496001728982, 'f1_score': 0.9304777594728171, 'conf_matrix': array([[4431,  242],
       [ 391, 4236]])}


In [40]:
import numpy as np
from keras.models import load_model
from keras import backend as K
from sklearn.metrics import accuracy_score

start_time = time.time()

# Load models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Make predictions on the test set
preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)

# Define weights for each model
weight1 = 0.3  # Adjust according to preference
weight2 = 0.7  # Adjust according to preference

# Convert weights to Keras tensors
weight1_tensor = K.variable(weight1)
weight2_tensor = K.variable(weight2)

# Combine predictions using weighted average
weighted_preds1 = preds1 * weight1_tensor
weighted_preds2 = preds2 * weight2_tensor
hybrid_preds = weighted_preds1 + weighted_preds2

# Evaluate hybrid model performance
hybrid_labels = K.eval(K.greater(hybrid_preds, 0.5)).astype(int)

cpu_use, mem_use = get_system_info()
pred_time=calculate_time(start_time)

hybrid_accuracy = accuracy_score(y_test, hybrid_labels)
print("Hybrid Model Accuracy:", hybrid_accuracy)

metrics = evaluate_performance("Weighted Average Hybrid .3 : .7 keras", y_test, hybrid_labels, mem_use, cpu_use, pred_time, prt=1)
print(metrics )
performance_data.append(metrics)

291/291 [==============================] - 1s 2ms/step
Hybrid Model Accuracy: 0.9319354838709677
Weighted Average Hybrid .3 : .7 keras Model Performance: 
Time 1710341414.6901925
Prediction time 1.8172407150268555
Memory Usage 13.1
CPU usage 56.1
Accuracy: 0.9319354838709677
Precision: 0.9459580169718624
Recall: 0.915496001728982
F1-score: 0.9304777594728171
Confusion Matrix:
[[4431  242]
 [ 391 4236]]
{'data': 'Weighted Average Hybrid .3 : .7 keras', 'time': 1710341414.6893024, 'pred time': 1.8172407150268555, 'memory usage': 13.1, 'cpu usage': 56.1, 'accuracy': 0.9319354838709677, 'precision': 0.9459580169718624, 'recall': 0.915496001728982, 'f1_score': 0.9304777594728171, 'conf_matrix': array([[4431,  242],
       [ 391, 4236]])}


In [41]:
start_time = time.time()

# Load the models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Define weights
weights = [0.15, 0.85]  # Modify this to your desired weights

# Define a function for prediction and combination
shallow_pred = shallow_model.predict(X_test)
deep_pred = deep_model.predict(X_test)
hybrid_preds = weights[0] * shallow_pred + weights[1] * deep_pred
hybrid_labels = (hybrid_preds > 0.5).astype(int)

pred_time=calculate_time(start_time)

cpu_use, mem_use = get_system_info()

metrics = evaluate_performance("Weighted Average Hybrid2 .15:.85", y_test, hybrid_labels, mem_use, cpu_use, pred_time, prt=1)
print(metrics)
performance_data.append(metrics)




291/291 [==============================] - 0s 2ms/step
Weighted Average Hybrid2 .15:.85 Model Performance: 
Time 1710341416.404095
Prediction time 1.6660354137420654
Memory Usage 13.1
CPU usage 57.6
Accuracy: 0.9343010752688172
Precision: 0.954093170511081
Recall: 0.9118219148476334
F1-score: 0.9324787269311525
Confusion Matrix:
[[4470  203]
 [ 408 4219]]
{'data': 'Weighted Average Hybrid2 .15:.85', 'time': 1710341416.402443, 'pred time': 1.6660354137420654, 'memory usage': 13.1, 'cpu usage': 57.6, 'accuracy': 0.9343010752688172, 'precision': 0.954093170511081, 'recall': 0.9118219148476334, 'f1_score': 0.9324787269311525, 'conf_matrix': array([[4470,  203],
       [ 408, 4219]])}


In [42]:
start_time = time.time()

# Load the models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Define weights
weights = [1, 1]  # Modify this to your desired weights

# Define a function for prediction and combination
shallow_pred = shallow_model.predict(X_test)
deep_pred = deep_model.predict(X_test)
hybrid_preds = weights[0] * shallow_pred + weights[1] * deep_pred
hybrid_labels = (hybrid_preds > 0.5).astype(int)

pred_time=calculate_time(start_time)

cpu_use, mem_use = get_system_info()


metrics = evaluate_performance("Weighted Average Hybrid2 1:1 ", y_test, hybrid_labels,  mem_use, cpu_use, pred_time, prt=1)
print(metrics)
performance_data.append(metrics)




291/291 [==============================] - 1s 2ms/step
Weighted Average Hybrid2 1:1  Model Performance: 
Time 1710341418.1509898
Prediction time 1.7041618824005127
Memory Usage 13.1
CPU usage 63.0
Accuracy: 0.8893548387096775
Precision: 0.8261421319796954
Recall: 0.9848714069591528
F1-score: 0.8985507246376812
Confusion Matrix:
[[3714  959]
 [  70 4557]]
{'data': 'Weighted Average Hybrid2 1:1 ', 'time': 1710341418.1502366, 'pred time': 1.7041618824005127, 'memory usage': 13.1, 'cpu usage': 63.0, 'accuracy': 0.8893548387096775, 'precision': 0.8261421319796954, 'recall': 0.9848714069591528, 'f1_score': 0.8985507246376812, 'conf_matrix': array([[3714,  959],
       [  70, 4557]])}


## Concatenate Hybrid

In [43]:
import tensorflow as tf
import numpy as np

start_time = time.time()
# Load the models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)


# Make predictions on test data
model1_pred = model1.predict(X_test)
model2_pred = model2.predict(X_test)

# Concatenate predictions
hybrid_preds = np.concatenate([model1_pred, model2_pred], axis=1)  # Change axis as needed

hybrid_labels = (hybrid_preds > 0.5).astype(int)

cpu_use, mem_use = get_system_info()
pred_time=calculate_time(start_time)

# Compare to individual models
individual_metrics = (
    model1.evaluate(X_test, y_test),
    model2.evaluate(X_test, y_test),
)

# Print individual model metrics
print("Individual Model Performance:")
for i, model in enumerate([model1, model2]):
  print(f"Model {i+1} Metrics:")
  print(individual_metrics[i])


mean_preds  = np.mean(hybrid_preds, axis=1)
hybrid_labels = (mean_preds > 0.5).astype(int)

print(hybrid_labels)
metrics = evaluate_performance("Concatenate  Hybrid ", y_test, hybrid_labels, mem_use, cpu_use, pred_time, prt=1)

print(metrics )
performance_data.append(metrics)


291/291 [==============================] - 1s 2ms/step - loss: 0.1416 - accuracy: 0.9345
Individual Model Performance:
Model 1 Metrics:
[0.2053688019514084, 0.9072043299674988]
Model 2 Metrics:
[0.14159196615219116, 0.9345161318778992]
[1 0 1 ... 1 1 0]
Concatenate  Hybrid  Model Performance: 
Time 1710341423.7521741
Prediction time 2.1127583980560303
Memory Usage 13.0
CPU usage 70.5
Accuracy: 0.9294623655913978
Precision: 0.9336099585062241
Recall: 0.9239247892803112
F1-score: 0.9287421247012818
Confusion Matrix:
[[4369  304]
 [ 352 4275]]
{'data': 'Concatenate  Hybrid ', 'time': 1710341423.751426, 'pred time': 2.1127583980560303, 'memory usage': 13.0, 'cpu usage': 70.5, 'accuracy': 0.9294623655913978, 'precision': 0.9336099585062241, 'recall': 0.9239247892803112, 'f1_score': 0.9287421247012818, 'conf_matrix': array([[4369,  304],
       [ 352, 4275]])}


## Maximum

In [44]:
start_time = time.time()

# Load models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Make predictions on the test set
preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)

# Take the maximum prediction value across models
max_preds = np.maximum(preds1, preds2)
max_labels = (max_preds > 0.5).astype(int)


pred_time=calculate_time(start_time)
cpu_use, mem_use = get_system_info()



# Evaluate hybrid model performance
accuracy = accuracy_score(y_test, max_labels)
print("Hybrid Model Accuracy:", accuracy)

metrics = evaluate_performance("Maximum  Hybrid ", y_test, max_labels, mem_use, cpu_use, pred_time, prt=1)
print(metrics)
performance_data.append(metrics)

291/291 [==============================] - 1s 2ms/step
Hybrid Model Accuracy: 0.9138709677419354
Maximum  Hybrid  Model Performance: 
Time 1710341426.123726
Prediction time 2.333055019378662
Memory Usage 12.9
CPU usage 64.2
Accuracy: 0.9138709677419354
Precision: 0.8850644122383253
Recall: 0.9502917657229306
F1-score: 0.916519020323085
Confusion Matrix:
[[4102  571]
 [ 230 4397]]
{'data': 'Maximum  Hybrid ', 'time': 1710341426.123603, 'pred time': 2.333055019378662, 'memory usage': 12.9, 'cpu usage': 64.2, 'accuracy': 0.9138709677419354, 'precision': 0.8850644122383253, 'recall': 0.9502917657229306, 'f1_score': 0.916519020323085, 'conf_matrix': array([[4102,  571],
       [ 230, 4397]])}


In [45]:
start_time = time.time()

# Load models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Make predictions on the test set
preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)

# Take the maximum prediction value across models
min_preds = np.minimum(preds1, preds2)
min_labels = (min_preds > 0.5).astype(int)

pred_time=calculate_time(start_time)
cpu_use, mem_use = get_system_info()


# Evaluate hybrid model performance
accuracy = accuracy_score(y_test, min_labels)
print("Hybrid Model Accuracy:", accuracy)
metrics = evaluate_performance("Minimum  Hybrid ", y_test, min_labels, mem_use, cpu_use, pred_time, prt=1)
print(metrics)
performance_data.append(metrics)

291/291 [==============================] - 1s 2ms/step
Hybrid Model Accuracy: 0.9278494623655914
Minimum  Hybrid  Model Performance: 
Time 1710341428.6251976
Prediction time 2.4566564559936523
Memory Usage 12.9
CPU usage 44.8
Accuracy: 0.9278494623655914
Precision: 0.9625818521983162
Recall: 0.8895612708018155
F1-score: 0.9246321464674828
Confusion Matrix:
[[4513  160]
 [ 511 4116]]
{'data': 'Minimum  Hybrid ', 'time': 1710341428.6250753, 'pred time': 2.4566564559936523, 'memory usage': 12.9, 'cpu usage': 44.8, 'accuracy': 0.9278494623655914, 'precision': 0.9625818521983162, 'recall': 0.8895612708018155, 'f1_score': 0.9246321464674828, 'conf_matrix': array([[4513,  160],
       [ 511, 4116]])}


## Add models

In [46]:
start_time = time.time()
# Load models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)

# Add the predictions of both models
sum_preds = preds1 + preds2

# Convert summed predictions to labels if necessary
# Assuming binary classification, you can use a threshold of 0.5
sum_labels = (sum_preds > 0.5).astype(int)

pred_time=calculate_time(start_time)
cpu_use, mem_use = get_system_info()

# Evaluate hybrid model performance
accuracy = accuracy_score(y_test, sum_labels)
print("Hybrid Model Accuracy:", accuracy)

metrics = evaluate_performance("Add Hybrid ", y_test, sum_labels, mem_use, cpu_use, pred_time, prt=1)

print(metrics)
performance_data.append(metrics)

291/291 [==============================] - 1s 2ms/step
Hybrid Model Accuracy: 0.8893548387096775
Add Hybrid  Model Performance: 
Time 1710341431.0726626
Prediction time 2.4020769596099854
Memory Usage 12.9
CPU usage 59.3
Accuracy: 0.8893548387096775
Precision: 0.8261421319796954
Recall: 0.9848714069591528
F1-score: 0.8985507246376812
Confusion Matrix:
[[3714  959]
 [  70 4557]]
{'data': 'Add Hybrid ', 'time': 1710341431.0725303, 'pred time': 2.4020769596099854, 'memory usage': 12.9, 'cpu usage': 59.3, 'accuracy': 0.8893548387096775, 'precision': 0.8261421319796954, 'recall': 0.9848714069591528, 'f1_score': 0.8985507246376812, 'conf_matrix': array([[3714,  959],
       [  70, 4557]])}


##Substract Hybrid

In [47]:
start_time = time.time()

# Load models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Make predictions on the test set
preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)

# Add the predictions of both models
sub_preds = preds1 - preds2
sub_preds = np.abs(sub_preds)

# Convert summed predictions to labels if necessary
# Assuming binary classification, you can use a threshold of 0.5
sub_labels = (sub_preds > 0.5).astype(int)

pred_time=calculate_time(start_time)
cpu_use, mem_use = get_system_info()


# Evaluate hybrid model performance
accuracy = accuracy_score(y_test, sub_labels)
print("Hybrid Model Accuracy:", accuracy)

metrics = evaluate_performance("Subtract Hybrid ", y_test, sum_labels,  mem_use, cpu_use, pred_time, prt=1)

print(metrics)
performance_data.append(metrics)

291/291 [==============================] - 1s 2ms/step
Hybrid Model Accuracy: 0.49838709677419357
Subtract Hybrid  Model Performance: 
Time 1710341433.6181457
Prediction time 2.488483190536499
Memory Usage 13.1
CPU usage 57.9
Accuracy: 0.8893548387096775
Precision: 0.8261421319796954
Recall: 0.9848714069591528
F1-score: 0.8985507246376812
Confusion Matrix:
[[3714  959]
 [  70 4557]]
{'data': 'Subtract Hybrid ', 'time': 1710341433.617036, 'pred time': 2.488483190536499, 'memory usage': 13.1, 'cpu usage': 57.9, 'accuracy': 0.8893548387096775, 'precision': 0.8261421319796954, 'recall': 0.9848714069591528, 'f1_score': 0.8985507246376812, 'conf_matrix': array([[3714,  959],
       [  70, 4557]])}


## Multiply

In [48]:
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score
start_time = time.time()

# Load models
model1 = load_model(shallow_model_hp_path)
model2 = load_model(deep_model_hp_path)

# Make predictions on the test set
preds1 = model1.predict(X_test)
preds2 = model2.predict(X_test)

# Multiply the predictions of both models
mul_preds = preds1 * preds2

# Convert multiplied predictions to labels if necessary
# Assuming binary classification, you can use a threshold of 0.5
mul_labels = (mul_preds > 0.5).astype(int)

pred_time=calculate_time(start_time)

cpu_use, mem_use = get_system_info()

# Evaluate hybrid model performance
accuracy = accuracy_score(y_test, mul_labels)
print("Hybrid Model Accuracy:", accuracy)

# Optionally, evaluate additional metrics
metrics = evaluate_performance("Multiply Hybrid", y_test, mul_labels, mem_use, cpu_use, pred_time, prt=1)

print(metrics)

# Optionally, append the performance metrics to a list
performance_data.append(metrics)


291/291 [==============================] - 1s 2ms/step
Hybrid Model Accuracy: 0.9193548387096774
Multiply Hybrid Model Performance: 
Time 1710341436.9848475
Prediction time 3.3118255138397217
Memory Usage 13.1
CPU usage 66.5
Accuracy: 0.9193548387096774
Precision: 0.9792336217552534
Recall: 0.8560622433542252
F1-score: 0.9135147601476015
Confusion Matrix:
[[4589   84]
 [ 666 3961]]
{'data': 'Multiply Hybrid', 'time': 1710341436.983642, 'pred time': 3.3118255138397217, 'memory usage': 13.1, 'cpu usage': 66.5, 'accuracy': 0.9193548387096774, 'precision': 0.9792336217552534, 'recall': 0.8560622433542252, 'f1_score': 0.9135147601476015, 'conf_matrix': array([[4589,   84],
       [ 666, 3961]])}


In [49]:
performance_data

[{'data': 'Weighted Average Hybrid .3 : .7 numpy',
  'time': 1710341412.8269904,
  'pred time': 1.8805198669433594,
  'memory usage': 13.1,
  'cpu usage': 22.1,
  'accuracy': 0.9319354838709677,
  'precision': 0.9459580169718624,
  'recall': 0.915496001728982,
  'f1_score': 0.9304777594728171,
  'conf_matrix': array([[4431,  242],
         [ 391, 4236]])},
 {'data': 'Weighted Average Hybrid .3 : .7 keras',
  'time': 1710341414.6893024,
  'pred time': 1.8172407150268555,
  'memory usage': 13.1,
  'cpu usage': 56.1,
  'accuracy': 0.9319354838709677,
  'precision': 0.9459580169718624,
  'recall': 0.915496001728982,
  'f1_score': 0.9304777594728171,
  'conf_matrix': array([[4431,  242],
         [ 391, 4236]])},
 {'data': 'Weighted Average Hybrid2 .15:.85',
  'time': 1710341416.402443,
  'pred time': 1.6660354137420654,
  'memory usage': 13.1,
  'cpu usage': 57.6,
  'accuracy': 0.9343010752688172,
  'precision': 0.954093170511081,
  'recall': 0.9118219148476334,
  'f1_score': 0.93247872693

In [50]:
# Convert performance_data to a pandas DataFrame
df = pd.DataFrame(performance_data)

# Save the DataFrame to a CSV file
df.to_csv(metrics_f, index=False)

print("Saved to", metrics_f)


Saved to /content/drive/MyDrive/ANN_Edge/unsw_data/final/ml/mlj1_metrics_hybrid_results.csv


Weighted Average Hybrid2 .15:.85" model stands out as it has the highest accuracy among all models. Additionally, it maintains a good balance between precision, recall, and F1 score. Therefore, selecting the "Weighted Average Hybrid2 .15:.85" model as the best hybrid model.

## Final Hybrid Model

In [51]:
from keras.models import Model
from keras.layers import Input, concatenate
from keras.models import load_model

# Load the models
shallow_model = load_model(shallow_model_hp_path)
deep_model = load_model(deep_model_hp_path)

# Define weights
weights = [0.15, 0.85]  # Modify this to your desired weights

# Define input layer (assuming the input shape is the same for both models)
input_layer = Input(shape=shallow_model.input_shape[1:])

# Get individual predictions from both models
shallow_pred = shallow_model(input_layer)
deep_pred = deep_model(input_layer)

# Combine predictions using weighted average
hybrid_pred = concatenate([shallow_pred * weights[0], deep_pred * weights[1]])

# Define the hybrid model
hybrid_model = Model(inputs=input_layer, outputs=hybrid_pred)

# Save the hybrid model
hybrid_model.save(final_hybrid_model_path)

print("Hybrid model saved as", final_hybrid_model_path)


Hybrid model saved as /content/drive/MyDrive/ANN_Edge/unsw_data/final/final_hybrid_model.keras


## Testing hybrid model

In [52]:

pred_time=calculate_time(start_time)
# Load the hybrid model
hybrid_model = load_model(final_hybrid_model_path)

# Evaluate the retrained model on test data
test_preds = hybrid_model.predict(X_test)

# Calculate average probabilities across classes
#test_labels = np.mean(test_preds, axis=1)

# Decide on label based on the average probabilities
test_labels = (test_preds > 0.5).astype(int)

#flatten  labels
flattened_test_labels = [1 if any(label) else 0 for label in test_labels]
test_labels=flattened_test_labels

cpu_use, mem_use = get_system_info()
pred_time=calculate_time(start_time)

# Calculate accuracy using the flattened labels
test_accuracy = accuracy_score(y_test,test_labels)

print("Retrained hybrid model accuracy on test data:", test_accuracy)

metrics = evaluate_performance(" Final Hybrid 0.15 : .85 concatenate ", y_test, test_labels, mem_use, cpu_use, pred_time, prt=1)

print(metrics)


291/291 [==============================] - 1s 2ms/step
Retrained hybrid model accuracy on test data: 0.9304301075268817
 Final Hybrid 0.15 : .85 concatenate  Model Performance: 
Time 1710341438.9323103
Prediction time 5.2433922290802
Memory Usage 13.1
CPU usage 60.3
Accuracy: 0.9304301075268817
Precision: 0.9691183404054691
Recall: 0.8884806570131835
F1-score: 0.9270492727477732
Confusion Matrix:
[[4542  131]
 [ 516 4111]]
{'data': ' Final Hybrid 0.15 : .85 concatenate ', 'time': 1710341438.9313765, 'pred time': 5.2433922290802, 'memory usage': 13.1, 'cpu usage': 60.3, 'accuracy': 0.9304301075268817, 'precision': 0.9691183404054691, 'recall': 0.8884806570131835, 'f1_score': 0.9270492727477732, 'conf_matrix': array([[4542,  131],
       [ 516, 4111]])}


## Inference

In [53]:

start_time = time.time()
hybrid_model=load_summary(final_hybrid_model_path)
n=100
preds=do_inference(hybrid_model, X_test,y_test, n, 0, 0)
#print(preds)
test_labels = [1 if any(label) else 0 for label in preds]
cpu_use, mem_use = get_system_info()
pred_time=calculate_time(start_time)


#print(test_labels)
metrics = evaluate_performance("inferencing  ", y_test[:n], test_labels, mem_use, cpu_use, pred_time, prt=1)

print(metrics)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 sequential_1 (Sequential)   (None, 1)                    11265     ['input_2[0][0]']             
                                                                                                  
 sequential_5 (Sequential)   (None, 1)                    49313     ['input_2[0][0]']             
                                                                                                  
 tf.math.multiply_2 (TFOpLa  (None, 1)                    0         ['sequential_1[0][0]']        
 mbda)                                                                                      

## inference with balanced classes

In [54]:
# Load the inference data
inference_data1 = pd.read_csv(inference_data_path1)
# Separate features (X_test) and target variable (y_test)
Xi_test_raw = inference_data1[features]
scaler = StandardScaler() # scaling
Xi_test = scaler.fit_transform(Xi_test_raw)
yi_test = inference_data1['label']  # Replace 'target_column_name' with the actual name of your target column
print(Xi_test.shape,  yi_test.shape)

start_time = time.time()
hybrid_model=load_model(final_hybrid_model_path)
n=100
preds=do_inference(hybrid_model, Xi_test,yi_test, n, 0, 0)
#print(preds)
test_labels = [1 if any(label) else 0 for label in preds]

cpu_use, mem_use = get_system_info()
pred_time=calculate_time(start_time)

#print(test_labels)
metrics = evaluate_performance("inferencing  ", yi_test[:n], test_labels, mem_use, cpu_use, pred_time, prt=1)


print(metrics)


(10000, 20) (10000,)
4/4 [==============================] - 0s 4ms/step
inferencing   Model Performance: 
Time 1710341440.3529084
Prediction time 0.6600790023803711
Memory Usage 13.1
CPU usage 55.9
Accuracy: 0.9
Precision: 1.0
Recall: 0.7916666666666666
F1-score: 0.8837209302325582
Confusion Matrix:
[[52  0]
 [10 38]]
{'data': 'inferencing  ', 'time': 1710341440.3522584, 'pred time': 0.6600790023803711, 'memory usage': 13.1, 'cpu usage': 55.9, 'accuracy': 0.9, 'precision': 1.0, 'recall': 0.7916666666666666, 'f1_score': 0.8837209302325582, 'conf_matrix': array([[52,  0],
       [10, 38]])}


## Inferencd with random data

In [55]:
# Load the inference data
inference_data2 = pd.read_csv(inference_data_path2)
# Separate features (X_test) and target variable (y_test)
Xi_test_raw = inference_data2[features]
# Preprocess the test data (e.g., scaling)
scaler = StandardScaler()
Xi_test = scaler.fit_transform(Xi_test_raw)
yi_test = inference_data2['label']  # Replace 'target_column_name' with the actual name of your target column
print(Xi_test.shape,  yi_test.shape)

start_time = time.time()

hybrid_model=load_model(final_hybrid_model_path)
n=1000
preds=do_inference(hybrid_model, Xi_test,yi_test, n, 0, 0)
#print(preds)
test_labels = [1 if any(label) else 0 for label in preds]


cpu_use, mem_use = get_system_info()
pred_time=calculate_time(start_time)

#print(test_labels)
metrics = evaluate_performance("inferencing  ", yi_test[:n], test_labels, mem_use, cpu_use, pred_time, prt=1)

print(metrics)

(10000, 20) (10000,)
32/32 [==============================] - 0s 2ms/step
inferencing   Model Performance: 
Time 1710341441.1497185
Prediction time 0.6896243095397949
Memory Usage 13.1
CPU usage 57.2
Accuracy: 0.93
Precision: 0.9783333333333334
Recall: 0.9114906832298136
F1-score: 0.9437299035369775
Confusion Matrix:
[[343  13]
 [ 57 587]]
{'data': 'inferencing  ', 'time': 1710341441.1491206, 'pred time': 0.6896243095397949, 'memory usage': 13.1, 'cpu usage': 57.2, 'accuracy': 0.93, 'precision': 0.9783333333333334, 'recall': 0.9114906832298136, 'f1_score': 0.9437299035369775, 'conf_matrix': array([[343,  13],
       [ 57, 587]])}


In [56]:
import datetime

# Get the current date and time
current_time = datetime.datetime.now()

# Format the current date and time
formatted_datetime = current_time.strftime("%Y-%m-%d %H:%M:%S")

# Print the formatted date and time
print("Formatted current date and time:", formatted_datetime)


Formatted current date and time: 2024-03-13 14:50:41
